In [1]:
# coding: utf-8

# In[1]:


# rdkit
from rdkit import Chem
from rdkit.Chem import AllChem, MACCSkeys, RDKFingerprint, Descriptors, rdMolDescriptors, Lipinski
from rdkit.Chem.AtomPairs import Pairs, Torsions
from rdkit.Avalon import pyAvalonTools
from rdkit.Chem.EState import Fingerprinter
from rdkit.Chem import rdFingerprintGenerator

# sklearn
from sklearn import svm
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, precision_score, \
    recall_score, cohen_kappa_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os

# typing
from typing import Union

# connecting to chembl through web client
from chembl_webresource_client.new_client import new_client

from datetime import datetime
import shutil

from datetime import datetime
import os
from concurrent.futures import ProcessPoolExecutor, as_completed

import shutil
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np
import time

from joblib import dump
import pickle

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import train_test_split

# In[2]:

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier



In [2]:
def get_smiles_from_drug_names(drug_list, new_client, return_type='list'):
    molecule = new_client.molecule
    smiles_list = []
    drug_names = []

    for drug in drug_list:
        res = molecule.search(drug)

        if res:
            smiles_list.append(res[0]['molecule_structures']['canonical_smiles'])
            drug_names.append(drug)

    # Check return type
    if return_type == 'dataframe':
        # Convert list to DataFrame
        smiles_df = pd.DataFrame({
            'drug_name': drug_names,
            'smiles': smiles_list
        })
        return smiles_df
    elif return_type == 'list':
        return smiles_list
    else:
        raise ValueError("Invalid return_type. Expected 'dataframe' or 'list'.")


# In[3]:


def data_training(path: str, label: Union[int, str]) -> pd.DataFrame:
    """Loads and combines CSV files from a directory, assigns class labels, 
    removes duplicates, and returns a DataFrame.

    Args:
        path (str): Directory path containing the CSV files.
        label (Union[int, str]): Label value to be assigned as class 1.

    Returns:
        pd.DataFrame: DataFrame containing the 'smiles' column and class labels.

    Raises:
        ValueError: If the label is not in the valid range.
    """
    # Number of CSV files in the directory
    number_of_labels = len(glob.glob(f'{path}/*.csv'))

    # Convert label to string and pad with zeros if necessary
    label_str = str(label).zfill(2)

    # Validate label
    if len(label_str) > 2 or not 0 <= int(label_str) <= number_of_labels:
        raise ValueError(f"The label must be between 0 and {number_of_labels}.")

    # Label file path
    label_file = f"{path}/00_label_{label_str}.csv"

    # List all CSV files in the directory
    csv_files = glob.glob(f'{path}/*.csv')

    # Remove the label file from the list
    csv_files.remove(label_file)

    # Load the label file into a DataFrame and assign class 1
    df_positive = pd.read_csv(label_file)[['smiles']]
    if not df_positive.empty:
        df_positive['y'] = 1

        # Load the other CSV files into a DataFrame and assign class 0
        df_negative = pd.concat([pd.read_csv(file)[['smiles']] for file in csv_files])
        df_negative['y'] = 0

        # Combine the two DataFrames
        df = pd.concat([df_positive, df_negative])

        # Remove duplicates
        df.drop_duplicates(subset='smiles', inplace=True)

    else:
        df = pd.DataFrame()

    return df


# In[4]:


def apply_descriptors_and_fp(df: pd.DataFrame,
                             label_column: str,
                             smiles_column: str,
                             fp_method: str,
                             use_fingerprint: bool = True,
                             use_descriptors: bool = True) -> Union[np.ndarray, np.ndarray]:
    """Applies descriptor and fingerprint methods to a DataFrame and returns scaled descriptors and labels as numpy arrays.

    Args:
        df (pd.DataFrame): DataFrame containing the SMILES strings and labels.
        label_column (str): Name of the label column.
        smiles_column (str): Name of the SMILES column.
        fp_method (str): Name of the fingerprint method to apply.
        use_descriptors (bool): Whether or not to include descriptors in the output.
        use_fingerprint (bool): Whether or not to include fingerprint in the output.

    Returns:
        X (np.ndarray): Descriptor and/or fingerprint values depending on use_descriptors and use_fingerprint flags.
        Y (np.ndarray): Labels.
    """
    descriptor_methods_dict = {
        'MolWt': Descriptors.MolWt,
        'HeavyAtomCount': Descriptors.HeavyAtomCount,
        'NumHDonors': Descriptors.NumHDonors,
        'NumHAcceptors': Descriptors.NumHAcceptors,
        'MolLogP': Descriptors.MolLogP,
        'TPSA': rdMolDescriptors.CalcTPSA,
        'NumRotatableBonds': Lipinski.NumRotatableBonds,
        'NumAromaticRings': Descriptors.NumAromaticRings,
        'NumAliphaticRings': Descriptors.NumAliphaticRings,
    }
    fp_methods_dict = {
        'RDKFingerprint': RDKFingerprint,
        'MorganFingerprint': lambda m: AllChem.GetMorganFingerprintAsBitVect(m, 2),
        'MACCSkeys': MACCSkeys.GenMACCSKeys,
        'AvalonFP': pyAvalonTools.GetAvalonFP,
        'Gobbi-Paalman': rdMolDescriptors.GetHashedTopologicalTorsionFingerprintAsBitVect,
    }

    # Validate the fingerprint method
    if fp_method not in fp_methods_dict:
        raise ValueError(f"Invalid fingerprint method. Choices are: {list(fp_methods_dict.keys())}")

    # Convert SMILES to RDKit molecules
    df['Molecule'] = df[smiles_column].apply(Chem.MolFromSmiles)

    # Apply the descriptor methods and scale them if use_descriptors is True
    if use_descriptors:
        for descriptor, method in descriptor_methods_dict.items():
            df[descriptor] = df['Molecule'].apply(method)
        scaler = MinMaxScaler()
        df[list(descriptor_methods_dict.keys())] = scaler.fit_transform(df[list(descriptor_methods_dict.keys())])

    # Apply the fingerprint method if use_fingerprint is True
    if use_fingerprint:
        df[fp_method] = df['Molecule'].apply(fp_methods_dict[fp_method])
        df[f'np_{fp_method}'] = df[fp_method].apply(
            lambda fingerprints: np.array(list(map(int, fingerprints.ToBitString()))))
        X_fp = np.stack(df[f'np_{fp_method}'].values)

    # Create numpy array for descriptor data if use_descriptors is True
    if use_descriptors and use_fingerprint:
        X_descr = df[list(descriptor_methods_dict.keys())].values
        # Concatenate along columns axis (assuming fingerprint is 1D)
        X = np.hstack((X_descr, X_fp))
    elif use_descriptors:
        X = df[list(descriptor_methods_dict.keys())].values
    elif use_fingerprint:
        X = X_fp
    else:
        raise ValueError("At least one of use_descriptors or use_fingerprint must be True")

    # Create numpy array for label data
    if label_column:
        Y = df[label_column].values
    else:
        Y = None

    return X, Y, df




In [3]:
# In[5]:


drug_list_to_exclude = [
    "Amodiaquine",
    "Crizotinib",
    "Mebhydrolin",
    "Harringtonin",
    "Fluphenazine",
    "Fingolimod",
    "Fendiline",
    "Dronedarone",
    "Perphenazine",
    "Pimavanserin",
    "Prochlorperazine",
    "Raloxifene",
    "Abemaciclib",
    "Mefloquine",
    "Revaprazan",
    "Melitracen",
    "Nelfinavir",
    "Nicardipine",
    "Nilotinib",
    "Olmutinib",
    "harringtonin",
    "Terconazole",
    "Thioridazine",
    "Thiothixene",
    "Tilorone-dihydrochloride",
    "Triflupromazine"
]

smiles_list = get_smiles_from_drug_names(drug_list_to_exclude, new_client, return_type='list')
smiles_list

# In[6]:


smiles_dataframe = get_smiles_from_drug_names(drug_list_to_exclude, new_client, return_type='dataframe')
smiles_dataframe

# In[ ]:


now = datetime.now()
folder_name = now.strftime("%Y_%b_%d") + str('_code_classifier')
folder_name = folder_name.lower()
if os.path.isdir(folder_name):
    shutil.rmtree(folder_name)
os.makedirs(folder_name)

df_label = pd.read_csv('label-all-new.csv', header=None)
df_label.reset_index(inplace=True)
df_label.rename(columns={'index': 'label', 0: 'location'}, inplace=True)
df_label = df_label[['label', 'location']]

# Fingerprint methods list
fp_method_list = ['RDKFingerprint']

# Create dictionary to store results
smiles_prediction_results = {}

# Results dictionary
result_dict = {}

start = time.time()
ml_methods = [
    {'name': 'Random Forest', 'classifier': RandomForestClassifier()},
    {'name': 'Logistic Regression', 'classifier': LogisticRegression()},
    {'name': 'Gradient Boosting', 'classifier': GradientBoostingClassifier()},
    {'name': 'Ada Boost', 'classifier': AdaBoostClassifier()},
#     {'name': 'Dummy', 'classifier': DummyClassifier()},
#     {'name': 'Passive Aggressive', 'classifier': PassiveAggressiveClassifier(max_iter=1000)},
    {'name': 'Perceptron', 'classifier': Perceptron()},
    {'name': 'SGD', 'classifier': SGDClassifier()},
    {'name': 'SVC', 'classifier': SVC()},
    {'name': 'Linear SVC', 'classifier': LinearSVC()},
]

number_of_files = len([f for f in os.listdir('final_label_to_train_LV') if os.path.isfile(os.path.join('final_label_to_train_LV', f))])

for label_location in range(number_of_files):
    print(label_location)
    df = data_training('final_label_to_train_LV', label_location)
    if not df.empty:
        df = df[~df['smiles'].isin(smiles_list)]
        location = df_label.loc[df_label['label'] == label_location, 'location'].values[0]
        for finger_print_method in fp_method_list:
            for descriptor_bool in [True,False]:
                for finger_print_bool in [True]:
                    if finger_print_bool or descriptor_bool:
                        X, Y, df = apply_descriptors_and_fp(df, 'y', 'smiles',
                                                            finger_print_method,
                                                            finger_print_bool,
                                                            descriptor_bool)
                        number_of_zeros = sum([1 for i in Y if i == 0])
                        number_of_ones = sum([1 for i in Y if i == 1])

                        zero_ratio = round(max(number_of_zeros, number_of_ones) / number_of_zeros)
                        one_ratio = round(max(number_of_zeros, number_of_ones) / number_of_ones)
                        class_weight = {0: zero_ratio, 1: one_ratio}

                        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, stratify=Y)

                        # Initialize the SVC with probability=True
                        for method in ml_methods:
                            print(f"Using {method['name']} method...")
                            clf = method['classifier']
                            if 'class_weight' in clf.get_params():
#                                 clf.set_params(class_weight=class_weight)
                                pass
                            if method['name'] == 'Logistic Regression':
                                clf.set_params(max_iter=500)
                            
                            clf.fit(X_train, y_train)

                            y_pred = clf.predict(X_test)

                            # Calculate probabilities for each class
                            if hasattr(clf, 'predict_proba'):
                                # Calculate probabilities for each class
                                probs = clf.predict_proba(X_test)
                                class0_probs = probs[:, 0]
                                class1_probs = probs[:, 1]
                            else:
                                # Set to 'NA' if predict_proba method does not exist
                                class0_probs = 'NA'
                                class1_probs = 'NA'

                            unique_key = f"{method['name']}_{label_location}_{finger_print_method}_{finger_print_bool}_{descriptor_bool}"
                            result_dict[unique_key] = {
                                'method': method['name'],
                                'location': location,
                                'finger_print_method': finger_print_method,
                                'finger_print_bool': finger_print_bool,
                                'descriptor_bool': descriptor_bool,
                                'accuracy': accuracy_score(y_test, y_pred),
                                'kappa': cohen_kappa_score(y_test, y_pred),
                                'f1_score': f1_score(y_test, y_pred, average='weighted'),
                                'f1_score_class_0': f1_score(y_test, y_pred, average=None)[0],
                                'f1_score_class_1': f1_score(y_test, y_pred, average=None)[1],
                                'precision': precision_score(y_test, y_pred, average='weighted'),
                                'recall': recall_score(y_test, y_pred, average='weighted'),
                                'label_1': np.count_nonzero(Y == 1),
                                'label_0': np.count_nonzero(Y == 0),
                                'class0_proba': np.mean(class0_probs) if class0_probs != 'NA' else 'NA',
                                'class1_proba': np.mean(class1_probs) if class1_probs != 'NA' else 'NA'
                            }
                            print(f1_score(y_test, y_pred, average=None)[0])
                            print(f1_score(y_test, y_pred, average=None)[1])
                            # Predict on smiles_list and store results
                            smiles_features = apply_descriptors_and_fp(smiles_dataframe, None, 'smiles',
                                                                       finger_print_method,
                                                                       finger_print_bool,
                                                                       descriptor_bool)[0]
                            smiles_pred = clf.predict(smiles_features)
                            if hasattr(clf, 'predict_proba'):
                                smiles_pred_proba = clf.predict_proba(smiles_features)  # Get probabilities

                            else:
                                smiles_pred_proba = np.full((smiles_pred.shape[0], 2), np.nan)
                            for i, prediction in enumerate(smiles_pred):
                                smile = f"{smiles_dataframe['smiles'].iloc[i]}_{label_location}_{finger_print_method}_{finger_print_bool}_{descriptor_bool}"
                                if smile in smiles_prediction_results:
                                    smiles_prediction_results[smile].append(
                                        [label_location, finger_print_method, finger_print_bool, descriptor_bool,
                                         smiles_pred[i], smiles_pred_proba[i, 0], smiles_pred_proba[i, 1]])
                                else:
                                    smiles_prediction_results[smile] = [label_location, finger_print_method,
                                                                        finger_print_bool, descriptor_bool,
                                                                        smiles_pred[i],
                                                                        smiles_pred_proba[i, 0],
                                                                        smiles_pred_proba[i, 1]]

                            folder_path = f'./{folder_name}/{label_location}_{location}/'
                            if not os.path.exists(folder_path):
                                os.makedirs(folder_path)
                            dump(clf,
                                 folder_path + f"{method['name']}_{finger_print_method}_{descriptor_bool}_{finger_print_bool}_model.joblib")
                            np.save(folder_path + f'{finger_print_method}_{descriptor_bool}_{finger_print_bool}_X.npy',
                                    X)
                            np.save(folder_path + f'{finger_print_method}_{descriptor_bool}_{finger_print_bool}_Y.npy',
                                    Y)

#                             np.save(
#                                 folder_path + f'{method["name"]}_{finger_print_method}_{descriptor_bool}_{finger_print_bool}_X_train.npy',
#                                 X_train)
#                             np.save(
#                                 folder_path + f'{method["name"]}_{finger_print_method}_{descriptor_bool}_{finger_print_bool}_Y_train.npy',
#                                 y_train)

#                             np.save(
#                                 folder_path + f'{method["name"]}_{finger_print_method}_{descriptor_bool}_{finger_print_bool}_X_test.npy',
#                                 X_test)
#                             np.save(
#                                 folder_path + f'{method["name"]}_{finger_print_method}_{descriptor_bool}_{finger_print_bool}_Y_test.npy',
#                                 y_test)

                            df.to_csv(
                                folder_path + f'{method["name"]}_{finger_print_method}_{descriptor_bool}_{finger_print_bool}_df.csv',
                                index=False)

    else:
        location = 'nan'
        unique_key = f"{label_location}_{location}"
        result_dict[unique_key] = {
            'method': 'NA',
            'location': location,
            'finger_print_method': 'NA',
            'finger_print_bool': 'NA',
            'kappa':'NA',
            'descriptor_bool': 'NA',
            'accuracy': 'NA',
            'f1_score': 'NA',
            'f1_score_class_0': 'NA',
            'f1_score_class_1': 'NA',
            'precision': 'NA',
            'recall': 'NA'
        }

df_result = pd.DataFrame.from_dict(result_dict, orient='index')
end = start - time.time()

# In[9]:


df_result.to_csv('final_result_classifier.csv', index=False)

with open('smiles_prediction_results_classifier', 'wb') as f:
    pickle.dump(smiles_prediction_results, f)

# In[10]:

0
1
Using Random Forest method...


/tmp/ipykernel_454702/165771129.py:146: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  'class0_proba': np.mean(class0_probs) if class0_probs != 'NA' else 'NA',
/tmp/ipykernel_454702/165771129.py:147: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  'class1_proba': np.mean(class1_probs) if class1_probs != 'NA' else 'NA'


0.8769405528209011
0.31865828092243187
Using Logistic Regression method...


/home/payam/.virtualenvs/nabi/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/tmp/ipykernel_454702/165771129.py:146: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  'class0_proba': np.mean(class0_probs) if class0_probs != 'NA' else 'NA',
/tmp/ipykernel_454702/165771129.py:147: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  'class1_proba': np.mean(class1_probs) if class1_probs != 'NA'

0.871853986551393
0.3530552861299709
Using Gradient Boosting method...


...

0.9548012962647109
0.28954423592493295
Using Ada Boost method...


/tmp/ipykernel_454702/165771129.py:146: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  'class0_proba': np.mean(class0_probs) if class0_probs != 'NA' else 'NA',
/tmp/ipykernel_454702/165771129.py:147: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  'class1_proba': np.mean(class1_probs) if class1_probs != 'NA' else 'NA'


0.9491293956981905
0.21164021164021166
Using Perceptron method...
0.9332628983976052
0.31956912028725315
Using SGD method...
0.9363876651982378
0.35650623885918004
Using SVC method...
0.9569083447332422
0.35051546391752575
Using Linear SVC method...


/home/payam/.virtualenvs/nabi/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/payam/.virtualenvs/nabi/lib/python3.10/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.9227722772277228
0.3700440528634361


In [4]:
df.empty

False

In [5]:
df_result = pd.DataFrame.from_dict(result_dict, orient='index')


In [11]:
df_result.to_csv('ZZ-diffmodels.csv', index = False)

In [ ]:
df_result

In [7]:
from datetime import datetime

# Get the current time and format it as a string
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f'reso_with_kappa_SVM_{timestamp}.csv'

# Save the dataframe without overwriting previous files
df_result.to_csv(filename, index=False)

In [8]:
drug_list_to_exclude = [
    "Amodiaquine",
    "Crizotinib",
    "Mebhydrolin",
    "Harringtonin",
    "Fluphenazine",
    "Fingolimod",
    "Fendiline",
    "Dronedarone",
    "Perphenazine",
    "Pimavanserin",
    "Prochlorperazine",
    "Raloxifene",
    "Abemaciclib",
    "Mefloquine",
    "Revaprazan",
    "Melitracen",
    "Nelfinavir",
    "Nicardipine",
    "Nilotinib",
    "Olmutinib",
    "harringtonin",
    "Terconazole",
    "Thioridazine",
    "Thiothixene",
    "Tilorone-dihydrochloride",
    "Triflupromazine"
]


smiles_dataframe = get_smiles_from_drug_names(drug_list_to_exclude, new_client, return_type='dataframe')
smiles_dataframe

,drug_name,smiles
0,Amodiaquine,CCN(CC)Cc1cc(Nc2ccnc3cc(Cl)ccc23)ccc1O
1,Crizotinib,C[C@@H](Oc1cc(-c2cnn(C3CCNCC3)c2)cnc1N)c1c(Cl)...
2,Mebhydrolin,CN1CCc2c(c3ccccc3n2Cc2ccccc2)C1
3,Harringtonin,COC(=O)C[C@@](O)(CCC(C)(C)O)C(=O)O[C@@H]1C(OC)...
4,Fluphenazine,OCCN1CCN(CCCN2c3ccccc3Sc3ccc(C(F)(F)F)cc32)CC1
5,Fingolimod,CCCCCCCCc1ccc(CCC(N)(CO)CO)cc1
6,Fendiline,CC(NCCC(c1ccccc1)c1ccccc1)c1ccccc1
7,Dronedarone,CCCCc1oc2ccc(NS(C)(=O)=O)cc2c1C(=O)c1ccc(OCCCN...
8,Perphenazine,OCCN1CCN(CCCN2c3ccccc3Sc3ccc(Cl)cc32)CC1
9,Pimavanserin,CC(C)COc1ccc(CNC(=O)N(Cc2ccc(F)cc2)C2CCN(C)CC2...


In [9]:
smiles_dataframe['smiles']

0                CCN(CC)Cc1cc(Nc2ccnc3cc(Cl)ccc23)ccc1O
1     C[C@@H](Oc1cc(-c2cnn(C3CCNCC3)c2)cnc1N)c1c(Cl)...
2                       CN1CCc2c(c3ccccc3n2Cc2ccccc2)C1
3     COC(=O)C[C@@](O)(CCC(C)(C)O)C(=O)O[C@@H]1C(OC)...
4        OCCN1CCN(CCCN2c3ccccc3Sc3ccc(C(F)(F)F)cc32)CC1
5                        CCCCCCCCc1ccc(CCC(N)(CO)CO)cc1
6                    CC(NCCC(c1ccccc1)c1ccccc1)c1ccccc1
7     CCCCc1oc2ccc(NS(C)(=O)=O)cc2c1C(=O)c1ccc(OCCCN...
8              OCCN1CCN(CCCN2c3ccccc3Sc3ccc(Cl)cc32)CC1
9     CC(C)COc1ccc(CNC(=O)N(Cc2ccc(F)cc2)C2CCN(C)CC2...
10    CN1CCN(CCCN2c3ccccc3Sc3ccc(Cl)cc32)CC1.O=S(=O)...
11    O=C(c1ccc(OCCN2CCCCC2)cc1)c1c(-c2ccc(O)cc2)sc2...
12    CCN1CCN(Cc2ccc(Nc3ncc(F)c(-c4cc(F)c5nc(C)n(C(C...
13    O[C@@H](c1cc(C(F)(F)F)nc2c(C(F)(F)F)cccc12)[C@...
14            Cc1nc(Nc2ccc(F)cc2)nc(N2CCc3ccccc3C2C)c1C
15                  CN(C)CCC=C1c2ccccc2C(C)(C)c2ccccc21
16    Cc1c(O)cccc1C(=O)N[C@@H](CSc1ccccc1)[C@H](O)CN...
17    COC(=O)C1=C(C)NC(C)=C(C(=O)OCCN(C)Cc2ccccc

In [16]:
for key in smiles_prediction_results:
    print(f"Length of list for key '{key}': {len(smiles_prediction_results[key])}")


Length of list for key 'CCN(CC)Cc1cc(Nc2ccnc3cc(Cl)ccc23)ccc1O_1_RDKFingerprint_True_True': 14
Length of list for key 'C[C@@H](Oc1cc(-c2cnn(C3CCNCC3)c2)cnc1N)c1c(Cl)ccc(F)c1Cl_1_RDKFingerprint_True_True': 14
Length of list for key 'CN1CCc2c(c3ccccc3n2Cc2ccccc2)C1_1_RDKFingerprint_True_True': 14
Length of list for key 'COC(=O)C[C@@](O)(CCC(C)(C)O)C(=O)O[C@@H]1C(OC)=C[C@]23CCCN2CCc2cc4c(cc2[C@@H]13)OCO4_1_RDKFingerprint_True_True': 22
Length of list for key 'OCCN1CCN(CCCN2c3ccccc3Sc3ccc(C(F)(F)F)cc32)CC1_1_RDKFingerprint_True_True': 14
Length of list for key 'CCCCCCCCc1ccc(CCC(N)(CO)CO)cc1_1_RDKFingerprint_True_True': 14
Length of list for key 'CC(NCCC(c1ccccc1)c1ccccc1)c1ccccc1_1_RDKFingerprint_True_True': 14
Length of list for key 'CCCCc1oc2ccc(NS(C)(=O)=O)cc2c1C(=O)c1ccc(OCCCN(CCCC)CCCC)cc1.Cl_1_RDKFingerprint_True_True': 14
Length of list for key 'OCCN1CCN(CCCN2c3ccccc3Sc3ccc(Cl)cc32)CC1_1_RDKFingerprint_True_True': 14
Length of list for key 'CC(C)COc1ccc(CNC(=O)N(Cc2ccc(F)cc2)C2CCN(

In [18]:
padding_value = None
max_length = max(len(lst) for lst in smiles_prediction_results.values())
padded_results = {k: v + [padding_value] * (max_length - len(v)) for k, v in smiles_prediction_results.items()}

In [20]:
df_smiles_prediction_results = pd.DataFrame.from_dict(padded_results, orient='index')  # or padded_results
df_smiles_prediction_results

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
CCN(CC)Cc1cc(Nc2ccnc3cc(Cl)ccc23)ccc1O_1_RDKFingerprint_True_True,1,RDKFingerprint,True,True,1,0.480000,0.520000,"[1, RDKFingerprint, True, True, 0, 0.794520807...","[1, RDKFingerprint, True, True, 0, 0.600654019...","[1, RDKFingerprint, True, True, 0, 0.674318326...",...,"[1, RDKFingerprint, True, True, 0, nan, nan]","[1, RDKFingerprint, True, True, 0, nan, nan]",None,None,None,None,None,None,None,None
C[C@@H](Oc1cc(-c2cnn(C3CCNCC3)c2)cnc1N)c1c(Cl)ccc(F)c1Cl_1_RDKFingerprint_True_True,1,RDKFingerprint,True,True,0,0.805000,0.195000,"[1, RDKFingerprint, True, True, 0, 0.970698517...","[1, RDKFingerprint, True, True, 0, 0.665462615...","[1, RDKFingerprint, True, True, 0, 0.675518301...",...,"[1, RDKFingerprint, True, True, 0, nan, nan]","[1, RDKFingerprint, True, True, 0, nan, nan]",None,None,None,None,None,None,None,None
CN1CCc2c(c3ccccc3n2Cc2ccccc2)C1_1_RDKFingerprint_True_True,1,RDKFingerprint,True,True,0,0.728500,0.271500,"[1, RDKFingerprint, True, True, 0, 0.760693774...","[1, RDKFingerprint, True, True, 0, 0.701247503...","[1, RDKFingerprint, True, True, 0, 0.506795055...",...,"[1, RDKFingerprint, True, True, 0, nan, nan]","[1, RDKFingerprint, True, True, 0, nan, nan]",None,None,None,None,None,None,None,None
COC(=O)C[C@@](O)(CCC(C)(C)O)C(=O)O[C@@H]1C(OC)=C[C@]23CCCN2CCc2cc4c(cc2[C@@H]13)OCO4_1_RDKFingerprint_True_True,1,RDKFingerprint,True,True,1,0.360000,0.640000,"[1, RDKFingerprint, True, True, 1, 0.36, 0.64]","[1, RDKFingerprint, True, True, 0, 0.817312424...","[1, RDKFingerprint, True, True, 0, 0.817312424...",...,"[1, RDKFingerprint, True, True, 0, 0.680129707...","[1, RDKFingerprint, True, True, 0, 0.680129707...","[1, RDKFingerprint, True, True, 1, nan, nan]","[1, RDKFingerprint, True, True, 1, nan, nan]","[1, RDKFingerprint, True, True, 0, nan, nan]","[1, RDKFingerprint, True, True, 0, nan, nan]","[1, RDKFingerprint, True, True, 0, nan, nan]","[1, RDKFingerprint, True, True, 0, nan, nan]","[1, RDKFingerprint, True, True, 0, nan, nan]","[1, RDKFingerprint, True, True, 0, nan, nan]"
OCCN1CCN(CCCN2c3ccccc3Sc3ccc(C(F)(F)F)cc32)CC1_1_RDKFingerprint_True_True,1,RDKFingerprint,True,True,0,0.595000,0.405000,"[1, RDKFingerprint, True, True, 0, 0.527573141...","[1, RDKFingerprint, True, True, 0, 0.589433826...","[1, RDKFingerprint, True, True, 0, 0.672241011...",...,"[1, RDKFingerprint, True, True, 0, nan, nan]","[1, RDKFingerprint, True, True, 1, nan, nan]",None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CC(C)N1CCN(c2ccc(OC[C@H]3CO[C@](Cn4cncn4)(c4ccc(Cl)cc4Cl)O3)cc2)CC1_33_RDKFingerprint_True_False,33,RDKFingerprint,True,False,0,0.860667,0.139333,"[33, RDKFingerprint, True, False, 0, 0.9374983...","[33, RDKFingerprint, True, False, 0, 0.8973594...","[33, RDKFingerprint, True, False, 0, 0.5112737...",...,"[33, RDKFingerprint, True, False, 0, nan, nan]","[33, RDKFingerprint, True, False, 1, nan, nan]",None,None,None,None,None,None,None,None
CSc1ccc2c(c1)N(CCC1CCCCN1C)c1ccccc1S2_33_RDKFingerprint_True_False,33,RDKFingerprint,True,False,0,0.975000,0.025000,"[33, RDKFingerprint, True, False, 0, 0.9472610...","[33, RDKFingerprint, True, False, 0, 0.9595287...","[33, RDKFingerprint, True, False, 0, 0.5102747...",...,"[33, RDKFingerprint, True, False, 0, nan, nan]","[33, RDKFingerprint, True, False, 0, nan, nan]",None,None,None,None,None,None,None,None
CN1CCN(CC/C=C2/c3ccccc3Sc3ccc(S(=O)(=O)N(C)C)cc32)CC1_33_RDKFingerprint_True_False,33,RDKFingerprint,True,False,0,0.960000,0.040000,"[33, RDKFingerprint, True, False, 0, 0.9699140...","[33, RDKFingerprint, True, False, 0, 0.9268728...","[33, RDKFingerprint, True, False, 0, 0.5120498...",...,"[33, RDKFingerprint, True, False, 0, nan, nan]","[33, RDKFingerprint, True, False, 0, nan, nan]",None,None,None,None,None,None,None,None
CCN(CC)CCOc1ccc2c(c1)C(=O)c1cc(OCCN(CC)CC)ccc1-2_33_RDKFingerprint_True_False,33,RDKFingerprint,True,False,0,0.809167,0.190833

In [15]:
df_smiles_prediction_results = pd.DataFrame.from_dict(smiles_prediction_results)

ValueError: All arrays must be of the same length

In [13]:
df_smiles_prediction_results.to_csv('ZZ-smiles_prediction_diffmodel_results.csv', index=False)

NameError: name 'df_smiles_prediction_results' is not defined

[1,
 'RDKFingerprint',
 True,
 True,
 1,
 0.36,
 0.64,
 [1, 'RDKFingerprint', True, True, 1, 0.36, 0.64],
 [1, 'RDKFingerprint', True, True, 0, 0.8173124243691264, 0.1826875756308736],
 [1, 'RDKFingerprint', True, True, 0, 0.8173124243691264, 0.1826875756308736],
 [1, 'RDKFingerprint', True, True, 0, 0.7533262564060454, 0.24667374359395455],
 [1, 'RDKFingerprint', True, True, 0, 0.7533262564060454, 0.24667374359395455],
 [1, 'RDKFingerprint', True, True, 0, 0.6801297071122772, 0.3198702928877229],
 [1, 'RDKFingerprint', True, True, 0, 0.6801297071122772, 0.3198702928877229],
 [1, 'RDKFingerprint', True, True, 1, nan, nan],
 [1, 'RDKFingerprint', True, True, 1, nan, nan],
 [1, 'RDKFingerprint', True, True, 0, nan, nan],
 [1, 'RDKFingerprint', True, True, 0, nan, nan],
 [1, 'RDKFingerprint', True, True, 0, nan, nan],
 [1, 'RDKFingerprint', True, True, 0, nan, nan],
 [1, 'RDKFingerprint', True, True, 0, nan, nan],
 [1, 'RDKFingerprint', True, True, 0, nan, nan]]

In [23]:
test1 = {'CCN(CC)Cc1cc(Nc2ccnc3cc(Cl)ccc23)ccc1O_1_RDKFingerprint_True_True': [1,
  'RDKFingerprint',
  True,
  True,
  1,
  0.48,
  0.52,
  [1,
   'RDKFingerprint',
   True,
   True,
   0,
   0.7945208079181986,
   0.20547919208180143],
  [1,
   'RDKFingerprint',
   True,
   True,
   0,
   0.6006540195113268,
   0.39934598048867315],
  [1, 'RDKFingerprint', True, True, 0, 0.674318326476439, 0.32568167352356103],
  [1, 'RDKFingerprint', True, True, 0, 'nan', 'nan'],
  [1, 'RDKFingerprint', True, True, 0, 'nan', 'nan'],
  [1, 'RDKFingerprint', True, True, 0, 'nan', 'nan'],
  [1, 'RDKFingerprint', True, True, 0, 'nan', 'nan']]}

In [25]:
len(test1['CCN(CC)Cc1cc(Nc2ccnc3cc(Cl)ccc23)ccc1O_1_RDKFingerprint_True_True'])

14

In [26]:
key = 'CCN(CC)Cc1cc(Nc2ccnc3cc(Cl)ccc23)ccc1O_1_RDKFingerprint_True_True'
first_seven_elements = test1[key][:7]  # This slices the list to get the first seven elements

# first_seven_elements now contains the first seven elements of the list
print(first_seven_elements)


[1, 'RDKFingerprint', True, True, 1, 0.48, 0.52]


In [27]:
test1

{'CCN(CC)Cc1cc(Nc2ccnc3cc(Cl)ccc23)ccc1O_1_RDKFingerprint_True_True': [1,
  'RDKFingerprint',
  True,
  True,
  1,
  0.48,
  0.52,
  [1,
   'RDKFingerprint',
   True,
   True,
   0,
   0.7945208079181986,
   0.20547919208180143],
  [1,
   'RDKFingerprint',
   True,
   True,
   0,
   0.6006540195113268,
   0.39934598048867315],
  [1, 'RDKFingerprint', True, True, 0, 0.674318326476439, 0.32568167352356103],
  [1, 'RDKFingerprint', True, True, 0, 'nan', 'nan'],
  [1, 'RDKFingerprint', True, True, 0, 'nan', 'nan'],
  [1, 'RDKFingerprint', True, True, 0, 'nan', 'nan'],
  [1, 'RDKFingerprint', True, True, 0, 'nan', 'nan']]}